# **Mount My Drive**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [44]:
%cd /content/drive/My Drive

/content/drive/My Drive


In [46]:
cd ProjectPM

[Errno 2] No such file or directory: 'ProjectPM'
/content/drive/My Drive/ProjectPM


In [47]:
!ls

colloquial-indonesian-lexicon.csv	 kamus_alay.csv
data_before_preprocessing.csv		 kata-dasar-indonesia.txt
data_kamus_kateglo.json			 kata_dasar_kbbi.csv
data_singkatan_dan_akronim_kateglo.json  kbba.txt
data_stem.csv				 tb_katadasar.sql
DBnews.db


# **PIP-ing**

In [48]:
pip install kbbi

In [49]:
pip install nltk

# **Import** **Libs**

In [0]:
#Importing Required Libs

import pandas as pd
import sqlite3 as sql
import re
import nltk
import glob
import numpy as np
import string
import logging as log
import kbbi
import os
import csv

from sklearn import datasets
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from kbbi import KBBI
from sqlite3 import Error

# **Urgent Section**

In [51]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [0]:
conn = sql.connect("/content/drive/My Drive/ProjectPM/DBnews.db")
dataset = pd.read_sql_query("SELECT * FROM t_comments", conn)

In [54]:
dataset

,comment_id,comment,baseline,correct_comment,comment_ptr
0,4207096,"WKWKWK, GAK SEKALIAN PAK, SELAWAT KE DPR... KAN BANYAK KORUPTOR DISANA",-1,None,1
1,4207096,Mantab.. ini br dakwah yg sejati.. kl d tempat umum mah dh biasa . Ini luar biasa.�,1,None,2
2,4207096,Salut buat Gus Miftah,1,None,3
3,4207096,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo \/ sunan kali jaga...smua ada cara demi kebaikan... asal jng ajk dg cara berjinah .... lakukan dg porsi dan sikon yg tepat,1,None,4
4,4207096,Ada adabnya sholawat. Lebih baik ajak ke majelis ilmu..,1,None,5
...,...,...,...,...,...
598092,4416660,212 nggak pernah ngadain acara beginian ya....terus program mereka dalam keagamaan apa ya....kalo mereka ngadain nya abis saya nulis ini...berarti mereka sebenernya cuman gerakan politik yg cari pembenaran dan kekuasaan doang dong,None,None,598093
598093,4416660,TBG... Sayank.. salah pilih kendraaan..�,None,None,598094
598094,4416660,Saya bersama TGB pilih Jokowi�,None,None,598095
598095,4416660,"Hijrah dari hoax ke non hoax, dari ngawur ke bener, dari saling menjelekkan ke saling puji dan hormat!",None,None,598096


In [0]:
all_comments = list(dataset['comment'])

In [57]:
all_comments

['WKWKWK, GAK SEKALIAN PAK, SELAWAT KE DPR... KAN BANYAK KORUPTOR DISANA',
 'Mantab.. ini br dakwah yg sejati.. kl d tempat umum mah dh biasa . Ini luar biasa.�',
 'Salut buat Gus Miftah',
 'yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo \\/ sunan kali jaga...smua ada cara demi kebaikan... asal jng ajk dg cara berjinah .... lakukan dg porsi dan sikon yg tepat',
 'Ada adabnya sholawat. Lebih baik ajak ke majelis ilmu..',
 'Setiap ulama punya jalan dan cara dakwah masing2.., jika kamu melihat kemunkaran lawan dengan tanganmu(kekuatanmu) jika tidak mampu lawan dengan mulutmu jika tidak mampu juga dengan hatimu itulah selemah lemahnya iman.., itu sudah menjadi kewajiban para ulama, kalo yg komen masih ada yg nyinyirin ulama berarti otaknya sudah disisipin politik..',
 'Umpanin nocannya yak...',
 'Itulah ceramah yg sebenarnya. ',
 'Besok bugil aja selawat',
 'Bedanya Gus Miftah dengan FPI  / HTI bagaikan surga dan neraka. Di surga penuh kasih, di neraka penuh p

In [59]:
# Case Folding: Lowercase
dataset['comment'] = dataset['comment'].str.lower()

# Remove Punctuation: Menghapus tanda baca
punctuation = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation))

dataset['comment'] = dataset['comment'].apply(lambda text: remove_punctuation(text))

# Non Alphabet Removal: Remove number and <?>
dataset['comment'] = dataset['comment'].str.replace(r'[^a-zA-Z ]\s?',r'', regex=True)
dataset.head(20)

,comment_id,comment,baseline,correct_comment,comment_ptr
0,4207096,wkwkwk gak sekalian pak selawat ke dpr kan banyak koruptor disana,-1,None,1
1,4207096,mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa,1,None,2
2,4207096,salut buat gus miftah,1,None,3
3,4207096,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo sunan kali jagasmua ada cara demi kebaikan asal jng ajk dg cara berjinah lakukan dg porsi dan sikon yg tepat,1,None,4
4,4207096,ada adabnya sholawat lebih baik ajak ke majelis ilmu,1,None,5
5,4207096,setiap ulama punya jalan dan cara dakwah masingjika kamu melihat kemunkaran lawan dengan tanganmukekuatanmu jika tidak mampu lawan dengan mulutmu jika tidak mampu juga dengan hatimu itulah selemah lemahnya iman itu sudah menjadi kewajiban para ulama kalo yg komen masih ada yg nyinyirin ulama berarti otaknya sudah disisipin politik,1,None,6
6,4207096,umpanin nocannya yak,0,None,7
7,4207096,itulah ceramah yg sebenarnya,0,None,8
8,4207096,besok bugil aja selawat,-1,None,9
9,4207096,bedanya gus miftah dengan fpi hti bagaikan surga dan neraka di surga penuh kasih di neraka penuh penghancuran,0,None,10


In [60]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
dataset['tokenized_comment'] = dataset.apply(lambda row: nltk.word_tokenize(row['comment']), axis=1)

In [62]:
dataset

,comment_id,comment,baseline,correct_comment,comment_ptr,tokenized_comment
0,4207096,wkwkwk gak sekalian pak selawat ke dpr kan banyak koruptor disana,-1,None,1,"[wkwkwk, gak, sekalian, pak, selawat, ke, dpr, kan, banyak, koruptor, disana]"
1,4207096,mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa,1,None,2,"[mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]"
2,4207096,salut buat gus miftah,1,None,3,"[salut, buat, gus, miftah]"
3,4207096,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo sunan kali jagasmua ada cara demi kebaikan asal jng ajk dg cara berjinah lakukan dg porsi dan sikon yg tepat,1,None,4,"[yg, nyinyir, gak, pernah, lihat, dan, baca, atw, mengetahui, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, kebaikan, asal, jng, ajk, dg, cara, berjinah, lakukan, dg, porsi, dan, sikon, yg, tepat]"
4,4207096,ada adabnya sholawat lebih baik ajak ke majelis ilmu,1,None,5,"[ada, adabnya, sholawat, lebih, baik, ajak, ke, majelis, ilmu]"
...,...,...,...,...,...,...
598092,4416660,nggak pernah ngadain acara beginian yaterus program mereka dalam keagamaan apa yakalo mereka ngadain nya abis saya nulis iniberarti mereka sebenernya cuman gerakan politik yg cari pembenaran dan kekuasaan doang dong,None,None,598093,"[nggak, pernah, ngadain, acara, beginian, yaterus, program, mereka, dalam, keagamaan, apa, yakalo, mereka, ngadain, nya, abis, saya, nulis, iniberarti, mereka, sebenernya, cuman, gerakan, politik, yg, cari, pembenaran, dan, kekuasaan, doang, dong]"
598093,4416660,tbg sayank salah pilih kendraaan,None,None,598094,"[tbg, sayank, salah, pilih, kendraaan]"
598094,4416660,saya bersama tgb pilih jokowi,None,None,598095,"[saya, bersama, tgb, pilih, jokowi]"
598095,4416660,hijrah dari hoax ke non hoax dari ngawur ke bener dari saling menjelekkan ke saling puji dan hormat,None,None,598096,"[hijrah, dari, hoax, ke, non, hoax, dari, ngawur, ke, bener, dari, saling, menjelekkan, ke, saling, puji, dan, hormat]"


In [0]:
pip install PySastrawi

     |████████████████████████████████| 215kB 8.2MB/s 


In [0]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

dataset['stem_comment'] = dataset['tokenized_comment'].apply(lambda x: [stemmer.stem(y) for y in x])

In [0]:
dataset

,comment_id,comment,baseline,correct_comment,comment_ptr,tokenized_comment,stem_comment
0,4207096,wkwkwk gak sekalian pak selawat ke dpr kan banyak koruptor disana,-1,None,1,"[wkwkwk, gak, sekalian, pak, selawat, ke, dpr, kan, banyak, koruptor, disana]","[wkwkwk, gak, sekali, pak, selawat, ke, dpr, kan, banyak, koruptor, sana]"
1,4207096,mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa,1,None,2,"[mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]","[mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]"
2,4207096,salut buat gus miftah,1,None,3,"[salut, buat, gus, miftah]","[salut, buat, gus, miftah]"
3,4207096,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo sunan kali jagasmua ada cara demi kebaikan asal jng ajk dg cara berjinah lakukan dg porsi dan sikon yg tepat,1,None,4,"[yg, nyinyir, gak, pernah, lihat, dan, baca, atw, mengetahui, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, kebaikan, asal, jng, ajk, dg, cara, berjinah, lakukan, dg, porsi, dan, sikon, yg, tepat]","[yg, nyinyir, gak, pernah, lihat, dan, baca, atw, tahu, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, baik, asal, jng, ajk, dg, cara, jah, laku, dg, porsi, dan, sikon, yg, tepat]"
4,4207096,ada adabnya sholawat lebih baik ajak ke majelis ilmu,1,None,5,"[ada, adabnya, sholawat, lebih, baik, ajak, ke, majelis, ilmu]","[ada, adab, sholawat, lebih, baik, ajak, ke, majelis, ilmu]"
...,...,...,...,...,...,...,...
598092,4416660,nggak pernah ngadain acara beginian yaterus program mereka dalam keagamaan apa yakalo mereka ngadain nya abis saya nulis iniberarti mereka sebenernya cuman gerakan politik yg cari pembenaran dan kekuasaan doang dong,None,None,598093,"[nggak, pernah, ngadain, acara, beginian, yaterus, program, mereka, dalam, keagamaan, apa, yakalo, mereka, ngadain, nya, abis, saya, nulis, iniberarti, mereka, sebenernya, cuman, gerakan, politik, yg, cari, pembenaran, dan, kekuasaan, doang, dong]","[nggak, pernah, ngadain, acara, begini, yaterus, program, mereka, dalam, agama, apa, yakalo, mereka, ngadain, nya, abis, saya, nulis, iniberarti, mereka, sebenernya, cuman, gera, politik, yg, cari, benar, dan, kuasa, doang, dong]"
598093,4416660,tbg sayank salah pilih kendraaan,None,None,598094,"[tbg, sayank, salah, pilih, kendraaan]","[tbg, sayank, salah, pilih, kendraaan]"
598094,4416660,saya bersama tgb pilih jokowi,None,None,598095,"[saya, bersama, tgb, pilih, jokowi]","[saya, sama, tgb, pilih, jokowi]"
598095,4416660,hijrah dari hoax ke non hoax dari ngawur ke bener dari saling menjelekkan ke saling puji dan hormat,None,None,598096,"[hijrah, dari, hoax, ke, non, hoax, dari, ngawur, ke, bener, dari, saling, menjelekkan, ke, saling, puji, dan, hormat]","[hijrah, dari, hoax, ke, non, hoax, dari, ngawur, ke, bener, dari, saling, jelek, ke, saling, puji, dan, hormat]"


In [64]:
print(dataset['stem_comment'])

0         [wkwkwk, gak, sekali, pak, selawat, ke, dpr, kan, banyak, koruptor, sana]                                                                                                                                                            
1         [mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]                                                                                                                                         
2         [salut, buat, gus, miftah]                                                                                                                                                                                                           
3         [yg, nyinyir, gak, pernah, lihat, dan, baca, atw, tahu, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, baik, asal, jng, ajk, dg, cara, jah, laku, dg, porsi, dan, sikon, yg, tepat]                             
4         [ada, adab, sholawat, lebih, b

In [65]:
print(dataset['stem_comment'][0][0])

wkwkwk


In [66]:
!ls

colloquial-indonesian-lexicon.csv	 kamus_alay.csv
data_before_preprocessing.csv		 kata-dasar-indonesia.txt
data_kamus_kateglo.json			 kata_dasar_kbbi.csv
data_singkatan_dan_akronim_kateglo.json  kbba.txt
data_stem.csv				 tb_katadasar.sql
DBnews.db


In [0]:
from tqdm import tqdm

In [0]:
for i in tqdm(range(len(dataset['stem_comment']))):
    dataset['stem_comment'][i] = ' '.join(dataset['stem_comment'][i])

print(dataset['stem_comment'])

  0%|          | 0/598097 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 598097/598097 [1:27:01<00:00, 114.54it/s]

0         wkwkwk gak sekali pak selawat ke dpr kan banyak koruptor sana                                                                                                                                        
1         mantab ini br dakwah yg sejati kl d tempat umum mah dh biasa ini luar biasa                                                                                                                          
2         salut buat gus miftah                                                                                                                                                                                
3         yg nyinyir gak pernah lihat dan baca atw tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jng ajk dg cara jah laku dg porsi dan sikon yg tepat                              
4         ada adab sholawat lebih baik ajak ke majelis ilmu                                                                                                             

In [0]:
dataset['stem_comment'].to_csv('data_stem.csv', index=False)

In [69]:
dfs = pd.read_csv('data_stem.csv')
dfs

,stem_comment
0,"['wkwkwk', 'gak', 'sekali', 'pak', 'selawat', 'ke', 'dpr', 'kan', 'banyak', 'koruptor', 'sana']"
1,"['mantab', 'ini', 'br', 'dakwah', 'yg', 'sejati', 'kl', 'd', 'tempat', 'umum', 'mah', 'dh', 'biasa', 'ini', 'luar', 'biasa']"
2,"['salut', 'buat', 'gus', 'miftah']"
3,"['yg', 'nyinyir', 'gak', 'pernah', 'lihat', 'dan', 'baca', 'atw', 'tahu', 'sejarah', 'dakwah', 'walisongo', 'sunan', 'kali', 'jagasmua', 'ada', 'cara', 'demi', 'baik', 'asal', 'jng', 'ajk', 'dg', 'cara', 'jah', 'laku', 'dg', 'porsi', 'dan', 'sikon', 'yg', 'tepat']"
4,"['ada', 'adab', 'sholawat', 'lebih', 'baik', 'ajak', 'ke', 'majelis', 'ilmu']"
...,...
598092,"['nggak', 'pernah', 'ngadain', 'acara', 'begini', 'yaterus', 'program', 'mereka', 'dalam', 'agama', 'apa', 'yakalo', 'mereka', 'ngadain', 'nya', 'abis', 'saya', 'nulis', 'iniberarti', 'mereka', 'sebenernya', 'cuman', 'gera', 'politik', 'yg', 'cari', 'benar', 'dan', 'kuasa', 'doang', 'dong']"
598093,"['tbg', 'sayank', 'salah', 'pilih', 'kendraaan']"
598094,"['saya', 'sama', 'tgb', 'pilih', 'jokowi']"
598095,"['hijrah', 'dari', 'hoax', 'ke', 'non', 'hoax', 'dari', 'ngawur', 'ke', 'bener', 'dari', 'saling', 'jelek', 'ke', 'saling', 'puji', 'dan', 'hormat']"


In [70]:
df_cil = pd.read_csv('colloquial-indonesian-lexicon.csv')
df_cil = df_cil.drop(['In-dictionary', 'context', 'category1', 'category2', 'category3'], axis = 1)
df_cil.head(5)

,slang,formal
0,woww,wow
1,aminn,amin
2,met,selamat
3,netaas,menetas
4,keberpa,keberapa


In [72]:
kbba = pd.read_csv('kbba.txt', delimiter = "\t", header = None, names = ["slang", "normal"])
kbba.head(5)

,slang,normal
0,7an,tujuan
1,@,di
2,ababil,abg labil
3,abis,habis
4,acc,accord


In [0]:
import json

In [75]:
k_dict = json.load(open('data_kamus_kateglo.json'))
df_kd_dict = pd.DataFrame(k_dict)
df_kd_dict = df_kd_dict.drop(['kelas', 'type'], axis = 1)
df_kd_dict.head(5)

,lema,nilai
0,-an,bt 1 hasil tindakan; 2 yang biasa dikenai tindakan; yang di-; 3 tempat; lokasi; 4 alat untuk mengukur atau menghitung; 5 hal atau cara; 6 kumpulan; gugusan; 7 yang mempunyai atau mengandung; 8 (cak) yang menyerupai; 9 sesuatu yang mempunyai sifat; 10 (cak) intensitas; 11 (cak) melakukan sesuatu yang pelaku atau tindakannya banyak (saling); 12 kegiatan yang berkenaan dengan pelaku atau tindakan yang banyak;
1,-anda,bt menyatakan hormat atau kekeluargaan;
2,-asi,bt → -isasi;
3,-da,bt → -anda;
4,-el-,bt 1 yang melakukan; 2 alat; 3 benda yang kelopak; 4 kumpulan; 5 terjadi berlaku atau melakukan;


In [76]:
df_kd_dict.shape

(72253, 2)

In [77]:
kaa = json.load(open('data_singkatan_dan_akronim_kateglo.json'))
df_kaa = pd.DataFrame.from_dict(kaa, orient = "index").reset_index()
df_kaa.columns = ['singkatan', 'bahasa_indonesia']
df_kaa.head(10)

,singkatan,bahasa_indonesia
0,a.d.,atas dasar
1,a.h.,None
2,a.i.,ad interim
3,a.l.,antara lain
4,a.m.,sebelum tengah hari
5,a.n.,atas nama
6,a.n.b.,atas nama beliau
7,a.p.,atas perintah
8,a.s.,alaihi salam
9,a/o,None


In [78]:
df_kaa.shape

(3389, 2)

In [0]:
tc_stem = dfs

In [80]:
tc_stem

,stem_comment
0,"['wkwkwk', 'gak', 'sekali', 'pak', 'selawat', 'ke', 'dpr', 'kan', 'banyak', 'koruptor', 'sana']"
1,"['mantab', 'ini', 'br', 'dakwah', 'yg', 'sejati', 'kl', 'd', 'tempat', 'umum', 'mah', 'dh', 'biasa', 'ini', 'luar', 'biasa']"
2,"['salut', 'buat', 'gus', 'miftah']"
3,"['yg', 'nyinyir', 'gak', 'pernah', 'lihat', 'dan', 'baca', 'atw', 'tahu', 'sejarah', 'dakwah', 'walisongo', 'sunan', 'kali', 'jagasmua', 'ada', 'cara', 'demi', 'baik', 'asal', 'jng', 'ajk', 'dg', 'cara', 'jah', 'laku', 'dg', 'porsi', 'dan', 'sikon', 'yg', 'tepat']"
4,"['ada', 'adab', 'sholawat', 'lebih', 'baik', 'ajak', 'ke', 'majelis', 'ilmu']"
...,...
598092,"['nggak', 'pernah', 'ngadain', 'acara', 'begini', 'yaterus', 'program', 'mereka', 'dalam', 'agama', 'apa', 'yakalo', 'mereka', 'ngadain', 'nya', 'abis', 'saya', 'nulis', 'iniberarti', 'mereka', 'sebenernya', 'cuman', 'gera', 'politik', 'yg', 'cari', 'benar', 'dan', 'kuasa', 'doang', 'dong']"
598093,"['tbg', 'sayank', 'salah', 'pilih', 'kendraaan']"
598094,"['saya', 'sama', 'tgb', 'pilih', 'jokowi']"
598095,"['hijrah', 'dari', 'hoax', 'ke', 'non', 'hoax', 'dari', 'ngawur', 'ke', 'bener', 'dari', 'saling', 'jelek', 'ke', 'saling', 'puji', 'dan', 'hormat']"


In [81]:
# df_kamus_alay = df_colloquial_indonesian_lexicon
df_kamus_alay = pd.read_csv('kamus_alay.csv')
df_kamus_alay

,slang,formal
0,woww,wow
1,aminn,amin
2,met,selamat
3,netaas,menetas
4,keberpa,keberapa
...,...,...
15001,gataunya,enggak taunya
15002,gtau,enggak tau
15003,gatau,enggak tau
15004,fans2,fan-fan


In [82]:
print(list(df_kamus_alay['slang']))

['woww', 'aminn', 'met', 'netaas', 'keberpa', 'eeeehhhh', 'kata2nyaaa', 'hallo', 'kaka', 'ka', 'daah', 'aaaaahhhh', 'yaa', 'smga', 'slalu', 'amiin', 'kk', 'trus', 'kk', 'sii', 'nyenengin', 'bgt', 'gemess', 'akuuu', 'jgn', 'yaa', 'udah', 'gitu', 'aja', 'gemesiin', 'menyenangkn', 'rb', 'akau', 'saranin', 'nemuin', 'yah', 'met', 'mnis2nya', 'gtu', 'tu', 'idolaa', 'bangett', 'kaakk', 'baaiikkk', 'telaatt', 'kaak', 'yg', 'lg', 'skarang', 'yg', 'sempet', 'karna', 'ngga', 'karna', 'mantep', 'loohhh', 'yeee', 'baguuuuus', 'sampe', 'ngikutin', 'gayany', 'ngikutin', 'lg', 'bnk', 'awaas', 'siip', 'abizzzz', 'kerrrrennnn', 'sayaa', 'pengen', 'kpan', 'bgt', 'sma', 'sdkit', 'bngt', 'lgu', 'bener2', 'kharismanya', 'aq', 'yg', 'jdi', 'lgi', 'gk', 'kek', 'lgunya', 'kpan', 'kereeeeennnn', 'nyamain', 'slalu', 'lakik', 'siippp', 'bkn', 'nyangka', 'bangettt', 'kereeen', 'wihhhhhh', 'tp', 'inget', 'bingitsss', 'dmn', 'bird', 'kaloo', 'gimna', 'tuuh', 'nyebur', 'yng', 'dbuat', 'kn', 'org', 'pingin', 'bgt', '

In [83]:
dataset['stem_comment']

0         [wkwkwk, gak, sekali, pak, selawat, ke, dpr, kan, banyak, koruptor, sana]                                                                                                                                                            
1         [mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]                                                                                                                                         
2         [salut, buat, gus, miftah]                                                                                                                                                                                                           
3         [yg, nyinyir, gak, pernah, lihat, dan, baca, atw, tahu, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, baik, asal, jng, ajk, dg, cara, jah, laku, dg, porsi, dan, sikon, yg, tepat]                             
4         [ada, adab, sholawat, lebih, b

In [84]:
for i in tqdm(range(len(dataset.head(1000)['stem_comment']))):
    for j in range(len(dataset.head(1000)['stem_comment'][i])):
        try:
            k_awal = dataset.head(1000)['stem_comment'][i][j]
            dataset.head(1000)['stem_comment'][i][j] = list(df_kamus_alay['formal'])[(list(df_kamus_alay['slang']).index(dataset.head(1000)['stem_comment'][i][j]))]
            k_akhir = dataset.head(1000)['stem_comment'][i][j] # kata yang diubah
            print(dataset.head(1000)['stem_comment'][i][j])
        except:
            pass
#             print(df_stem['stem_comment'][i][j])


  0%|          | 2/1000 [00:00<01:00, 16.47it/s]

enggak
baru
yang
kalau
ada
deh
yang
enggak
atau



  0%|          | 4/1000 [00:00<01:06, 15.09it/s]

jangan
dengan
dengan
yang



  1%|          | 10/1000 [00:00<01:04, 15.40it/s]

...

enggak
enggak
tidak
kapan
tidak
juga
karena
tidak



100%|█████████▉| 997/1000 [01:03<00:00, 20.53it/s]

yang
yang



100%|██████████| 1000/1000 [01:03<00:00, 15.79it/s]

yang


In [85]:
dataset.head(1000)['stem_comment']

0      [wkwkwk, enggak, sekali, pak, selawat, ke, dpr, kan, banyak, koruptor, sana]                                                                                                                                               
1      [mantab, ini, baru, dakwah, yang, sejati, kalau, ada, tempat, umum, mah, deh, biasa, ini, luar, biasa]                                                                                                                     
2      [salut, buat, gus, miftah]                                                                                                                                                                                                 
3      [yang, nyinyir, enggak, pernah, lihat, dan, baca, atau, tahu, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, baik, asal, jangan, ajk, dengan, cara, jah, laku, dengan, porsi, dan, sikon, yang, tepat]
4      [ada, adab, sholawat, lebih, baik, ajak, ke, majelis, ilmu]                          

In [86]:
for i in tqdm(range(len(dataset.head(1000)['stem_comment']))):
    dataset.head(1000)['stem_comment'][i] = ' '.join(dataset.head(1000)['stem_comment'][i])

print(dataset.head(1000)['stem_comment'])


  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)

100%|██████████| 1000/1000 [00:01<00:00, 798.23it/s]

0      [wkwkwk, enggak, sekali, pak, selawat, ke, dpr, kan, banyak, koruptor, sana]                                                                                                                                               
1      [mantab, ini, baru, dakwah, yang, sejati, kalau, ada, tempat, umum, mah, deh, biasa, ini, luar, biasa]                                                                                                                     
2      [salut, buat, gus, miftah]                                                                                                                                                                                                 
3      [yang, nyinyir, enggak, pernah, lihat, dan, baca, atau, tahu, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, baik, asal, jangan, ajk, dengan, cara, jah, laku, dengan, porsi, dan, sikon, yang, tepat]
4      [ada, adab, sholawat, lebih, baik, ajak, ke, majelis, ilmu]                          

In [87]:
dataset.head(1000)['stem_comment']

0      [wkwkwk, enggak, sekali, pak, selawat, ke, dpr, kan, banyak, koruptor, sana]                                                                                                                                               
1      [mantab, ini, baru, dakwah, yang, sejati, kalau, ada, tempat, umum, mah, deh, biasa, ini, luar, biasa]                                                                                                                     
2      [salut, buat, gus, miftah]                                                                                                                                                                                                 
3      [yang, nyinyir, enggak, pernah, lihat, dan, baca, atau, tahu, sejarah, dakwah, walisongo, sunan, kali, jagasmua, ada, cara, demi, baik, asal, jangan, ajk, dengan, cara, jah, laku, dengan, porsi, dan, sikon, yang, tepat]
4      [ada, adab, sholawat, lebih, baik, ajak, ke, majelis, ilmu]                          

In [0]:
dataset.head(1000)['stem_comment'].to_csv('data_final_1000data.csv', index=False)

In [93]:
df_final = pd.read_csv('data_final_1000data.csv')
df_final

,stem_comment
0,"['wkwkwk', 'enggak', 'sekali', 'pak', 'selawat', 'ke', 'dpr', 'kan', 'banyak', 'koruptor', 'sana']"
1,"['mantab', 'ini', 'baru', 'dakwah', 'yang', 'sejati', 'kalau', 'ada', 'tempat', 'umum', 'mah', 'deh', 'biasa', 'ini', 'luar', 'biasa']"
2,"['salut', 'buat', 'gus', 'miftah']"
3,"['yang', 'nyinyir', 'enggak', 'pernah', 'lihat', 'dan', 'baca', 'atau', 'tahu', 'sejarah', 'dakwah', 'walisongo', 'sunan', 'kali', 'jagasmua', 'ada', 'cara', 'demi', 'baik', 'asal', 'jangan', 'ajk', 'dengan', 'cara', 'jah', 'laku', 'dengan', 'porsi', 'dan', 'sikon', 'yang', 'tepat']"
4,"['ada', 'adab', 'sholawat', 'lebih', 'baik', 'ajak', 'ke', 'majelis', 'ilmu']"
...,...
995,"['bagus', 'jangan', 'ikut', 'orang', 'jahat', 'saudarapapua', 'rasa', 'bangun']"
996,"['apa', 'kata', 'gerinda', 'cs', 'ya', 'hayo', 'apa']"
997,"['karena', 'papu', 'maju']"
998,"['mbah', 'mien', 'langsung', 'kejangkejang']"


In [94]:
df_ff = pd.read_csv('data_final_1000data_fixed.csv')
df_ff

,stem_comment
0,wkwkwk enggak sekali pak selawat ke dpr kan banyak koruptor sana
1,mantab ini baru dakwah yang sejati kalau ada tempat umum mah deh biasa ini luar biasa
2,salut buat gus miftah
3,yang nyinyir enggak pernah lihat dan baca atau tahu sejarah dakwah walisongo sunan kali jagasmua ada cara demi baik asal jangan ajk dengan cara jah laku dengan porsi dan sikon yang tepat
4,ada adab sholawat lebih baik ajak ke majelis ilmu
...,...
995,bagus jangan ikut orang jahat saudarapapua rasa bangun
996,apa kata gerinda cs ya hayo apa
997,karena papu maju
998,mbah mien langsung kejangkejang
